In [76]:
import os
import detector as det
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Script for running analysis on all of Xin's data he passed along to me

### Specify path where the data is located

In [65]:
root = "/net/share/dfco222/data/Xins_CaSparkData/NewData_clipped/"

### Perform preprocessing and spark detection/analysis

In [68]:
### create dictionary to store results
wholeDataSetSparks = {}

### loop through directory and analyze each image. These must be .tif files!!
for name in os.listdir(root):
    print name
    if name[-4:] == '.tif':
        ### Performs computationally expensive task of smoothing the data using large and small mean kernels
        ###  NOTE: This is very computationally intensive and you may want to consider doing this individually for each image
        ## Comment this out once this has been performed for the first time. The data is saved for future use.
        det.smoothRawDataAndSave(root+name)
        ##
        
        ### Performs the actual detection and analysis of the sparks
        wholeDataSetSparks[name] = det.detectAndAnalyzeSparks_robust(root+name)

21.tif


detector.py:542: RuntimeWarning: invalid value encountered in divide
  transientImg = smallSmoothed.astype(np.float64) / largeSmoothed.astype(np.float64)
detector.py:561: RuntimeWarning: invalid value encountered in greater
  caSparks = timeGrad.copy() > thresholdValue


('Spark Number:', '1')
('\tTime to Peak:', 8.0, '[ms]')
('\tDistance to Outer Sarcolemma:', 0.479263850725286, '[microns]')
('\tSpark Amplitude:', 1.0957014879599452, '[AU]')
('\tDuration Half Maximum:', 28.0, '[ms]')
('\tStandard Deviation of Fitted Gaussian:', 0.44752954887123486)
('Spark Number:', '8')
('\tTime to Peak:', 12.0, '[ms]')
('\tDistance to Outer Sarcolemma:', 3.3548469550770017, '[microns]')
('\tSpark Amplitude:', 1.2370336564073316, '[AU]')
('\tDuration Half Maximum:', 26.0, '[ms]')
('\tStandard Deviation of Fitted Gaussian:', 1.4916416032997286)
('Final Spark Count:', 2)
20.tif
('Spark Number:', '31')
('\tTime to Peak:', 14.0, '[ms]')
('\tDistance to Outer Sarcolemma:', 0.6390184676337146, '[microns]')
('\tSpark Amplitude:', 1.3038153485861876, '[AU]')
('\tDuration Half Maximum:', 30.0, '[ms]')
('\tStandard Deviation of Fitted Gaussian:', 0.49299680983629873)
('Spark Number:', '40')
('\tTime to Peak:', 14.0, '[ms]')
('\tDistance to Outer Sarcolemma:', 3.195092338168573

### Turn data into arrays to more easily store in panda's dataframe object

In [69]:
sparkID = []
distanceToOuterSarcolemma = []
timeToPeak = []
sparkAmplitude = []
durationHalfMaximum = []
standardDeviationFittedGaussian = []
for fileName, sparkDict in wholeDataSetSparks.items():
    print fileName
    for sparkNum, subDict in sparkDict.items():
        sparkID.append(fileName[:-4] + "_"+str(sparkNum))
        distanceToOuterSarcolemma.append(subDict['dist_to_sarcolemma'])
        timeToPeak.append(subDict['time_to_peak'])
        sparkAmplitude.append(subDict['spark_amplitude'])
        durationHalfMaximum.append(subDict['duration_half_max'])
        standardDeviationFittedGaussian.append(subDict['sigma'])

7.tif
15.tif
16.tif
18.tif
11.tif
6.tif
9.tif
14.tif
4.tif
19.tif
12.tif
21.tif
8.tif
2.tif
5.tif
1.tif
13.tif
3.tif
20.tif
17.tif
10.tif


### Create DataFrame Object

In [73]:
data = np.asarray([
    sparkID,
    distanceToOuterSarcolemma,
    timeToPeak,
    sparkAmplitude,
    durationHalfMaximum,
    standardDeviationFittedGaussian,
])
data = np.transpose(data)

columns = ['Spark ID','Distance to Outer Sarcolemma [um]','Time to Peak [ms]','Spark Amplitude [AU]','Duration at Half Maximum [ms]','Fitted Gaussian Standard Deviation [um]']

df = pd.DataFrame(data,columns=columns)

In [81]:
printDF = False
if printDF:
    print df

### Write DataFrame to .csv

In [75]:
writeMe = True
if writeMe:
    df.to_csv(path_or_buf='./fullDatasetAnalysis.csv')